# Data Analysis of the Chicago Dataset

In this Jupyter Notebook we will analyze a bike sharing dataset of the city Chicago of the year 2019. We will also include the hourly weather data of the city during the same time. In the first part we will visualize the data and try to develop a deep understanding of the user types and recurring bahavioral patterns. In the second part we use this knowledge to develop a predicition model which forecasts future demand.

## Getting started: importing the data and getting it in the right format

At first we import all of the required libraries in this notebook. In this case we import *NumPy, Pandas, Matplotlib and Seaborn*. 


In [26]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

Next we import the datasets. This is in perticular the dataset of the city Chicago of the year 2019 and the associated weather data. We do this with the library Pandas, therefore both datasets have now the datatype "Pandas Dataframe" ([documentation](https://pandas.pydata.org/docs/reference/frame.html)).

In [2]:
chicago_set = pd.read_csv('chicago_2019.csv')
weather_set = pd.read_csv('weather_hourly_chicago.csv')

We use the commands head() and info() to get a feeling for the data

In [3]:
chicago_set.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,2019-01-01 00:04:37,2019-01-01 00:11:07,199,84,Wabash Ave & Grand Ave,Milwaukee Ave & Grand Ave,2167,Subscriber
1,2019-01-01 00:08:13,2019-01-01 00:15:34,44,624,State St & Randolph St,Dearborn St & Van Buren St (*),4386,Subscriber
2,2019-01-01 00:13:23,2019-01-01 00:27:12,15,644,Racine Ave & 18th St,Western Ave & Fillmore St (*),1524,Subscriber
3,2019-01-01 00:13:45,2019-01-01 00:43:28,123,176,California Ave & Milwaukee Ave,Clark St & Elm St,252,Subscriber
4,2019-01-01 00:14:52,2019-01-01 00:20:56,173,35,Mies van der Rohe Way & Chicago Ave,Streeter Dr & Grand Ave,1170,Subscriber


In [11]:
chicago_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818004 entries, 0 to 3818003
Data columns (total 9 columns):
 #   Column              Dtype          
---  ------              -----          
 0   start_time          datetime64[ns] 
 1   end_time            datetime64[ns] 
 2   start_station_id    int64          
 3   end_station_id      int64          
 4   start_station_name  object         
 5   end_station_name    object         
 6   bike_id             int64          
 7   user_type           object         
 8   duration            timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(3), timedelta64[ns](1)
memory usage: 262.2+ MB


So we know that we have a dataset with 3.8 million entries and we have multiple columns, two columns for the time (*start_time and end_time*), four columns for the location (*start_station_id, end_station_id and the respective names*) and the *bike_id* and *user_type*. 

In [12]:
weather_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  43788 non-null  object 
 1   max_temp   43788 non-null  float64
 2   min_temp   43788 non-null  float64
 3   precip     43790 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


We first investigate the time data we got. Therefore, we need to transform the data type of the columns "*start_time*" and "*end_time*" from String to Panda Datetime. This allows us in the following code snippet to add another column to our dataframe which we call *duration*. This is the difference between the end_time and the start_time. 

In [27]:
chicago_set['start_time'] = pd.to_datetime(chicago_set['start_time'])
chicago_set['end_time'] = pd.to_datetime(chicago_set['end_time'])

chicago_set['duration'] = chicago_set['end_time'] - chicago_set['start_time']
chicago_set.describe()

,start_station_id,end_station_id,bike_id,duration
count,3.818004e+06,3.818004e+06,3.818004e+06,3818004
mean,2.016659e+02,2.026363e+02,3.380252e+03,0 days 00:24:10.451397903
std,1.560798e+02,1.562388e+02,1.902511e+03,0 days 08:17:32.263421746
min,1.000000e+00,1.000000e+00,1.000000e+00,-1 days +23:03:38
25%,7.700000e+01,7.700000e+01,1.727000e+03,0 days 00:06:51
50%,1.740000e+02,1.740000e+02,3.451000e+03,0 days 00:11:49
75%,2.890000e+02,2.910000e+02,5.046000e+03,0 days 00:21:24
max,6.730000e+02,6.730000e+02,6.946000e+03,123 days 01:20:22


We calculated the duration of the rides and see that the maximum duration of a ride was over 123 days. We can assume that there a other cases where the data is inconclusive, therefore we will clean our dataset and delete those wrong outliners.

## Cleaning up our data

In the next step we count the rides between specific dates (first and last of a month) to get an overview how many rides take place per month. Furthermore, we'll use the same tactic between the times to find out during what times our services is getting used the most.

In [29]:
import datetime
chicago_set[chicago_set['duration'] >= datetime.timedelta(days=1)].sample(n=5)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,duration
3369931,2019-10-19 17:28:24,2019-10-26 09:50:34,255,97,Indiana Ave & Roosevelt Rd,Field Museum,1277,Customer,6 days 16:22:10
1838306,2019-07-22 10:03:26,2019-07-24 05:32:56,110,118,Dearborn St & Erie St,Sedgwick St & North Ave,1644,Customer,1 days 19:29:30
881276,2019-05-23 16:56:48,2019-05-29 11:02:48,435,373,Kedzie Ave & Roosevelt Rd,Kedzie Ave & Chicago Ave,449,Subscriber,5 days 18:06:00
361216,2019-03-30 16:34:32,2019-04-01 13:43:48,90,35,Millennium Park,Streeter Dr & Grand Ave,4120,Customer,1 days 21:09:16
1459869,2019-06-30 02:16:48,2019-07-01 07:44:29,118,455,Sedgwick St & North Ave,Maplewood Ave & Peterson Ave,2678,Customer,1 days 05:27:41


In [31]:
chicago_set[chicago_set['duration'] <= datetime.timedelta(days=0)].sample(n=5)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,duration
3498428,2019-11-03 01:43:21,2019-11-03 01:09:56,632,133,Clark St & Newport St,Kingsbury St & Kinzie St,4141,Customer,-1 days +23:26:35
3498439,2019-11-03 01:51:59,2019-11-03 01:03:02,460,238,Clark St & Bryn Mawr Ave,Wolcott (Ravenswood) Ave & Montrose Ave,2920,Subscriber,-1 days +23:11:03
3498447,2019-11-03 01:58:17,2019-11-03 01:08:27,301,52,Clark St & Schiller St,Michigan Ave & Lake St,6133,Subscriber,-1 days +23:10:10
3498445,2019-11-03 01:55:33,2019-11-03 01:01:52,109,320,900 W Harrison St,Loomis St & Lexington St,5059,Subscriber,-1 days +23:06:19
3498433,2019-11-03 01:46:01,2019-11-03 01:10:44,373,498,Kedzie Ave & Chicago Ave,California Ave & Fletcher St,6329,Customer,-1 days +23:24:43


In [30]:
#rides_per_month = pd.DataFrame({'end_dates':pd.date_range(start=end_dates.datetime(2019,1,1),end=dt.datetime(2019,31,12))})
#chicago_set['end_time'].groupby(['dt.month']).agg({'count'})
# <=pd.Timestamp(2019,1,31)




KeyError: 'dt.month'

In [33]:
fleet_size = len(chicago_set['bike_id'].unique())


#capacity_per_hour -> connect the fleet_size with the time to predict at what times in which areas we need to do something


6017


In the following part we try to visualize our data set. We want to make clear which time factory like months, time or weekday has influence on the demand. Furthermore, we try to find patterns of the duration and the time the service is getting used.

In [ ]:
rides_per_month = {'January',
'February', 'March', 'April','May','June','July', 'August','September', 'October','November','December'}

